In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from multiprocessing.pool import ThreadPool
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import numpy as np

In [48]:
data_match = pd.read_csv('/Users/swayam/Documents/Projects/Data_Mining/Data/Team_Strength_Match_Info.csv')
data_delivery = pd.read_csv('/Users/swayam/Documents/Projects/Data_Mining/Data/Delivery_Info.csv')

In [69]:
data_match = data_match.replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
data_delivery = data_delivery.replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
data_match = data_match.replace('Delhi Daredevils','Delhi Capitals')
data_delivery = data_delivery.replace('Delhi Daredevils','Delhi Capitals')
data_match = data_match.replace('Pune Warriors', 'Pune Warriors India')
data_delivery = data_delivery.replace('Pune Warriors', 'Pune Warriors India')

In [4]:
data_match.head(5)

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,result,DL_Method,Winner,Win_By_Runs,Win_By_Wicket,Man_Of_The_Match,Venue,Umpire1,Umpire2,Umpire3,Total_Balls,Team1_Strength,Team2_Strength
0,1,2013,5/19/13,Pune Warriors India,Delhi Capitals,Pune Warriors India,bat,Normal,0,Pune Warriors India,38.0,NaN,LJ Wright,Subrata Roy Sahara Stadium,NJ Llong,SJA Taufel,NaN,250,237.500000,242.250000
1,2,2015,4/15/15,Kings XI Punjab,Delhi Capitals,Kings XI Punjab,bat,Normal,0,Delhi Capitals,NaN,5.0,MA Agarwal,Maharashtra Cricket Association Stadium,CB Gaffaney,K Srinath,NaN,253,654.750000,1410.500000
2,3,2011,5/14/11,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Normal,1,Royal Challengers Bangalore,NaN,4.0,CH Gayle,M Chinnaswamy Stadium,RE Koertzen,RB Tiffin,NaN,159,579.000000,385.375000
3,4,2017,5/7/17,Kings XI Punjab,Gujarat Lions,Gujarat Lions,field,Normal,0,Gujarat Lions,NaN,6.0,DR Smith,"Punjab Cricket Association IS Bindra Stadium, ...",A Nand Kishore,VK Sharma,NaN,244,237.500000,183.214286
4,5,2010,4/17/10,Royal Challengers Bangalore,Mumbai Indians,Royal Challengers Bangalore,field,Normal,0,Mumbai Indians,57.0,NaN,R McLaren,M Chinnaswamy Stadium,HDPK Dharmasena,SJA Taufel,NaN,248,285.166667,420.625000


In [5]:
data_delivery.head(5)

,Match_ID,Innings,Batting_Team,Bowling_Team,Over,Ball,Batsman,Non_Striker,Bowler,Is_Super_Over,...,Bye_Runs,Legbye_Runs,Noball_Runs,Penalty_Runs,Batsman_Runs,Extras,Total_Runs,Player_Dismissed,Dismissal_Type,Fielder
0,1,1,Pune Warriors,Delhi Capitals,1,1,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Pune Warriors,Delhi Capitals,1,2,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
2,1,1,Pune Warriors,Delhi Capitals,1,3,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Pune Warriors,Delhi Capitals,1,4,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Pune Warriors,Delhi Capitals,1,5,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN


## Label Encoding

In [6]:
le = preprocessing.LabelEncoder()

### 1. Teams Encoding

In [7]:
Team1 = list(set(data_match.Team1))
le.fit(Team1)
data_match.Team1 = le.transform(data_match.Team1)
data_match.Team2 = le.transform(data_match.Team2)
data_match.Toss_Winner = le.transform(data_match.Toss_Winner)

### 2. Toss Encoding

In [8]:
toss_decision = list(set(data_match.Toss_Decision))
le.fit(toss_decision)
data_match.Toss_Decision = le.transform(data_match.Toss_Decision)

### 3. Venue Encoding 

In [9]:
venue = list(set(data_match.Venue))
le.fit(venue)
data_match.Venue = le.transform(data_match.Venue)

### 4. Umpire Encoding

In [10]:
s1 = set(data_match.Umpire1)
s2 = set(data_match.Umpire2)
umpire = list(s1.union(s2))
le.fit(umpire)
data_match.Umpire1 = le.transform(data_match.Umpire1)
data_match.Umpire2 = le.transform(data_match.Umpire2)

In [11]:
data_match

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,result,DL_Method,Winner,Win_By_Runs,Win_By_Wicket,Man_Of_The_Match,Venue,Umpire1,Umpire2,Umpire3,Total_Balls,Team1_Strength,Team2_Strength
0,1,2013,5/19/13,8,2,8,0,Normal,0,Pune Warriors India,38.0,NaN,LJ Wright,32,31,46,NaN,250,237.500000,242.250000
1,2,2015,4/15/15,4,2,4,0,Normal,0,Delhi Capitals,NaN,5.0,MA Agarwal,17,14,26,NaN,253,654.750000,1410.500000
2,3,2011,5/14/11,11,6,11,1,Normal,1,Royal Challengers Bangalore,NaN,4.0,CH Gayle,14,36,35,NaN,159,579.000000,385.375000
3,4,2017,5/7/17,4,3,3,1,Normal,0,Gujarat Lions,NaN,6.0,DR Smith,22,1,54,NaN,244,237.500000,183.214286
4,5,2010,4/17/10,11,7,11,1,Normal,0,Mumbai Indians,57.0,NaN,R McLaren,14,19,46,NaN,248,285.166667,420.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,5/12/16,12,2,2,1,Normal,0,Delhi Capitals,NaN,7.0,CH Morris,24,24,29,NaN,237,295.416667,433.000000
752,753,2016,5/25/16,12,6,6,1,Normal,0,Sunrisers Hyderabad,22.0,NaN,MC Henriques,8,29,13,NaN,244,253.214286,188.187500
753,754,2012,4/5/12,6,2,2,1,Normal,0,Delhi Capitals,NaN,8.0,IK Pathan,7,40,19,NaN,142,398.625000,529.166667
754,755,2014,5/7/14,4,0,0,1,Normal,0,Kings XI Punjab,44.0,NaN,GJ Maxwell,0,19,33,NaN,248,253.937500,264.000000


## Feature Engineering

### Creating Day, Month and Day_of_Week 

In [12]:
data_match['Date'] = pd.to_datetime(data_match['Date'], infer_datetime_format=True, errors='coerce')
data_match.dtypes
#Days of week 0-Monday 1-Tuesday 2-Wednesday 3-Thursday 4-Friday 5-Saturday 6-Sunday
data_match['Day_Of_Week'] = data_match['Date'].dt.weekday

#Date format mm dd year
data_match['Month'] = data_match['Date'].dt.month
data_match['Date'] = data_match['Date'].dt.day

### Average Balls Delivered by a team in a season

In [100]:
data = data_delivery.groupby('Match_ID')
result = pd.DataFrame()
for ID, value in data:
    team1 = data_match[data_match.Match_ID == ID].Team1.values[0]
    team2 = data_match[data_match.Match_ID == ID].Team2.values[0]
    team1_balls_delivered = value[value.Bowling_Team == team1].shape[0]
    team2_balls_delivered = value[value.Bowling_Team == team2].shape[0]
    temp = pd.DataFrame({'Team1_Balls_Delivered': team1_balls_delivered, 'Team2_Balls_Delivered': team2_balls_delivered}, index = [ID - 1])
    result = pd.concat([result, temp], sort = True)

data_match = 

,Team1_Balls_Delivered,Team2_Balls_Delivered
0,127,123
1,127,126
2,80,79
3,121,123
4,126,122
...,...,...
751,112,125
752,120,124
753,68,74
754,123,125


In [89]:
value

,Match_ID,Innings,Batting_Team,Bowling_Team,Over,Ball,Batsman,Non_Striker,Bowler,Is_Super_Over,...,Bye_Runs,Legbye_Runs,Noball_Runs,Penalty_Runs,Batsman_Runs,Extras,Total_Runs,Player_Dismissed,Dismissal_Type,Fielder
0,1,1,Pune Warriors India,Delhi Capitals,1,1,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Pune Warriors India,Delhi Capitals,1,2,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
2,1,1,Pune Warriors India,Delhi Capitals,1,3,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Pune Warriors India,Delhi Capitals,1,4,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Pune Warriors India,Delhi Capitals,1,5,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,2,Delhi Capitals,Pune Warriors India,20,1,UT Yadav,S Nadeem,AB Dinda,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
246,1,2,Delhi Capitals,Pune Warriors India,20,3,S Nadeem,UT Yadav,AB Dinda,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
247,1,2,Delhi Capitals,Pune Warriors India,20,3,S Nadeem,UT Yadav,AB Dinda,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
248,1,2,Delhi Capitals,Pune Warriors India,20,5,S Nadeem,UT Yadav,AB Dinda,0,...,0,0,0,0,0,0,0,S Nadeem,caught,AD Mathews


In [64]:
team1

'Pune Warriors India'

## Dimensionality Reduction

Since there are no 3rd Umpire we are going to drop that column

In [13]:
data_match = data_match.drop(['Umpire3'],  axis = 1)

Since we trying to predict the number of balls, result, Man_Of_The_Match of the match wont play a factor because we cant know the outcome before hand 

In [14]:
data_match = data_match.drop(['result', 'Winner', 'Man_Of_The_Match', 'Win_By_Runs', 'Win_By_Wicket'],  axis = 1)

In [15]:
data_match

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,DL_Method,Venue,Umpire1,Umpire2,Total_Balls,Team1_Strength,Team2_Strength,Day_Of_Week,Month
0,1,2013,19,8,2,8,0,0,32,31,46,250,237.500000,242.250000,6,5
1,2,2015,15,4,2,4,0,0,17,14,26,253,654.750000,1410.500000,2,4
2,3,2011,14,11,6,11,1,1,14,36,35,159,579.000000,385.375000,5,5
3,4,2017,7,4,3,3,1,0,22,1,54,244,237.500000,183.214286,6,5
4,5,2010,17,11,7,11,1,0,14,19,46,248,285.166667,420.625000,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,12,12,2,2,1,0,24,24,29,237,295.416667,433.000000,3,5
752,753,2016,25,12,6,6,1,0,8,29,13,244,253.214286,188.187500,2,5
753,754,2012,5,6,2,2,1,0,7,40,19,142,398.625000,529.166667,3,4
754,755,2014,7,4,0,0,1,0,0,19,33,248,253.937500,264.000000,2,5


#### Recursive feature elimination

In [16]:
X = data_match.drop(['Total_Balls'], axis = 1)
y = data_match['Total_Balls']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#X_train = X[X.Season != 2019]
#y_train = y[data_match.Season != 2019]

#X_test = X[X.Season == 2019]
#y_test = y[data_match.Season == 2019]

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
selector = RFE(reg, 5, step=1)
selector = selector.fit(X_train, y_train)
rank = selector.ranking_

index = np.where(rank == 1)
index

(array([ 3,  6,  7, 13, 14]),)

In [18]:
X_train.columns[index]

Index(['Team1', 'Toss_Decision', 'DL_Method', 'Day_Of_Week', 'Month'], dtype='object')

## Prediction Models

In [19]:
X = data_match.drop(['Total_Balls'], axis = 1)
y = data_match['Total_Balls']
X

,Match_ID,Season,Date,Team1,Team2,Toss_Winner,Toss_Decision,DL_Method,Venue,Umpire1,Umpire2,Team1_Strength,Team2_Strength,Day_Of_Week,Month
0,1,2013,19,8,2,8,0,0,32,31,46,237.500000,242.250000,6,5
1,2,2015,15,4,2,4,0,0,17,14,26,654.750000,1410.500000,2,4
2,3,2011,14,11,6,11,1,1,14,36,35,579.000000,385.375000,5,5
3,4,2017,7,4,3,3,1,0,22,1,54,237.500000,183.214286,6,5
4,5,2010,17,11,7,11,1,0,14,19,46,285.166667,420.625000,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,752,2016,12,12,2,2,1,0,24,24,29,295.416667,433.000000,3,5
752,753,2016,25,12,6,6,1,0,8,29,13,253.214286,188.187500,2,5
753,754,2012,5,6,2,2,1,0,7,40,19,398.625000,529.166667,3,4
754,755,2014,7,4,0,0,1,0,0,19,33,253.937500,264.000000,2,5


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#X_train = X[X.Season != 2019]
#y_train = y[data_match.Season != 2019]

#X_test = X[X.Season == 2019]
#y_test = y[data_match.Season == 2019]

In [23]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_predict = reg.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print('MSE of Linear Regression: ', error)
print('RMSE of Linear Regression: ', math.sqrt(error))

MSE of Linear Regression:  207.75834824685822
RMSE of Linear Regression:  14.413824899965249


In [29]:
def SVR(X_train, y_train):
    from sklearn.svm import SVR
    clf = SVR(kernel = 'poly', gamma='scale', C=1.0, epsilon=0.2)
    fit = clf.fit(X_train, y_train)
    
    
    return fit

In [30]:
SVR_fit = SVR(X_train, y_train)
y_predict = SVR_fit.predict(X_test)
error = mean_squared_error(y_test, y_predict)
print('MSE of SVR: ',error)
print('RMSE of SVR: ',math.sqrt(error))

MSE of SVR:  249.62450274144746
RMSE of SVR:  15.799509572814197
